In [1]:
#All neccesary classes for project

#general
import seaborn as sns
import scipy.stats as ss
import pandas as pd
import numpy as np
import os
import pickle
import time
import sys

#for preprocessing
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import tensorflow as tf

#for machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, reciprocal
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

#for evaluation
from sklearn.metrics import confusion_matrix

from sklearn.metrics import f1_score

#for utility packages
from Utilities.utilities import import_data
from Utilities.utilities import DataFrameSelector
from Utilities.utilities import CategoricalEncoder
from Utilities.utilities import display_scores
from Utilities.utilities import pipeline_transform
from Utilities.utilities import reset_graph
from Utilities.models import DNN_Model
from Utilities.models import cross_val_score_dnn
from functools import partial

#image manipulation
from PIL import Image as PI
from resizeimage import resizeimage
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from keras.utils.data_utils import get_file
import imagenet_helper_files.vgg_preprocessing
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.nets import resnet_v2
import imagenet_helper_files.pnasnet as nas

#Import Custom Functions
from Utilities.model_builder import get_image
from Utilities.model_builder import get_file_lists
from Utilities.model_builder import parse_record
from Utilities.model_builder import get_batch
from Utilities.model_builder import build_iterator
from Utilities.model_builder import build_dataset
from Utilities.bounded_model_builder import build_bounded_iterator
from Utilities.bounded_model_builder import build_bounded_iterator_points
from Utilities.model_builder import get_values_imagenet
from Utilities.model_builder import get_values_bounded
from Utilities.model_builder import get_values_bounded_points
from Utilities.models import log_dir_build
from Utilities.utilities import generate_image
from Utilities.utilities import generate_image_array
from Utilities.cell_net_predictor import Binary_Categorical_Predictor
from Utilities.build_image_data_notebook import process_dataset

from Utilities.utilities import get_ground_truth_string
from Utilities.utilities import find_new_imagenet_ground_truth_int
from Utilities.cell_net_predictor import Binary_Categorical_Predictor
from Utilities.build_image_data_notebook import process_dataset

from tan.tan_util import get_tan_nll as tan
from tan.tan_util import get_tan_nll_cond as tan_cond

Using TensorFlow backend.


# Load the Data

Here we will load the training and validation data in order to do training

In [2]:
#Build tf_records
#Set Variables
validation_directory = "D:/Machine_Learning/Datasets/airplane_vs_horse/validation"
train_directory = "D:/Machine_Learning/Datasets/airplane_vs_horse/train"
output_directory = "D:/Machine_Learning/Datasets/airplane_vs_horse/tf_records"
labels_file = "D:/Machine_Learning/Datasets/airplane_vs_horse/labels.txt"

num_threads = 2
num_shards = 2

In [ ]:
#make validation records
process_dataset('validation', validation_directory, num_shards, labels_file, num_threads, output_directory)

#make validation records
process_dataset('train', train_directory, num_shards, labels_file, num_threads, output_directory)

In [3]:
#Import TFRecords for both Training and Testing of the Dat
#Use the build_image_data.py to create these sets from your data
class_file = open(labels_file,'r')
class_list = class_file.read().split('\n')

train_list, val_list = get_file_lists(output_directory)
labels = class_list
labels

['airplane', 'horse']

In [4]:
validation_directory_all = "D:/Machine_Learning/Datasets/ten_class_classifier/validation"
train_directory_all = "D:/Machine_Learning/Datasets/ten_class_classifier/train"
output_directory_all = "D:/Machine_Learning/Datasets/ten_class_classifier/tf_records"
labels_file_all = "D:/Machine_Learning/Datasets/ten_class_classifier/labels.txt"


class_file_all = open(labels_file_all,'r')
class_list_all = class_file_all.read().split('\n')
train_list_all, val_list_all = get_file_lists(output_directory_all)
labels_all = class_list_all
labels_all

['airplane',
 'horse',
 'book',
 'cake',
 'car',
 'cat',
 'dog',
 'laptop',
 'pizza',
 'zebra']

In [ ]:
#make validation records
process_dataset('validation', validation_directory_all, num_shards, labels_file_all, num_threads, output_directory_all)

#make validation records
process_dataset('train', train_directory_all, num_shards, labels_file_all, num_threads, output_directory_all)

In [5]:
#Import imagenet files

with open('D:/AI/models/out_of_set_net_v2/image_net_dict.txt','r') as inf:
    image_net_dict_file = eval(inf.read())

old_image_net = open('D:/AI/models/out_of_set_net_v2/image_net_dict_2.txt').read().split('\n')
    
validation_array = open('D:/Machine_Learning/Devkits/Image_Net/ILSVRC2013_devkit/data/ILSVRC2013_clsloc_validation_ground_truth.txt').read().split()

In [ ]:
#Build a workflow to extract the data 
reset_graph()

filename = tf.placeholder(tf.string, shape=[None], name="tf_records")
batch_size = tf.placeholder(tf.int64, shape=[], name= "Batch_Size")
num_epochs = tf.placeholder(tf.int64, shape=[], name= "Num_epochs")
training = tf.placeholder_with_default(True, shape=(), name = 'training')
handle = tf.placeholder(tf.string, shape=[], name="Dataset")

training_set = build_dataset(True, filename, batch_size, num_epochs, num_parallel_calls=8)
val_set = build_dataset(False, filename, batch_size, num_epochs, num_parallel_calls=8)

train_iterator = training_set.make_initializable_iterator()
val_iterator = val_set.make_initializable_iterator()



iterator = tf.data.Iterator.from_string_handle(
    handle, training_set.output_types, training_set.output_shapes)
next_element = iterator.get_next()
X, y, file = next_element

X

In [ ]:
class_0 = tf.where(tf.equal(y,0))
class_1 = tf.where(tf.equal(y,1))

c0_X=tf.gather_nd(X,class_0)
c1_X=tf.gather_nd(X,class_1)

c0_y=tf.gather_nd(y,class_0)
c1_y=tf.gather_nd(y,class_1)

X_cond_0 = tf.cond(training, lambda: c0_X, lambda: X)
X_cond_1 = tf.cond(training, lambda: c1_X, lambda: X)

output = tf.cond(tf.equal(0,0), lambda: tf.gather_nd(X, tf.where(tf.equal(y,0))),lambda: tf.gather_nd(X, tf.where(tf.equal(y,1)))  )

In [ ]:
#Test pulling a piece of data out of the set to ensure that records were created properly
with tf.Session() as sess:
    #saver2.restore(sess, landmark_ml_model)
    sess.run(train_iterator.initializer, feed_dict={filename: train_list, batch_size: 10, num_epochs:1})
    train_handle = sess.run(train_iterator.string_handle())
    
    
    X_cond_out_0, X_cond_out_1 = sess.run([X_cond_0, X_cond_1], feed_dict={handle: train_handle,training: True})

  

In [ ]:
out[0].shape

In [ ]:
X_val = single_file[0][0]
Y_val = single_file[0][1]
filename = single_file[0][2]
X_val.shape

In [ ]:
x_val = X_val.reshape(331,331,3)

from matplotlib import pyplot as plt
plt.imshow(x_val, interpolation='nearest')
plt.axis('off')
plt.show()


In [ ]:
labels_all[Y_val[0]]

In [ ]:
filename

# Build the Neural Network on top of Large Nas-Net

Here we will build the Nas-Net and then stack our own network on top

In [6]:

#import pnas_net model

#Nasnet Model Location
nas_net_model = 'D:/AI/models/pnas_net/model.ckpt'

out_of_set_net_logs = 'D:/AI/models/out_of_set_net_v2/logs'
model_path = log_dir_build(out_of_set_net_logs, "out_of_set_v2_tan")
out_of_set_net_model_class = 'D:/AI/models/out_of_set_net_v2/final_model/' + 'out_of_set_net_class'
out_of_set_net_model_ae = 'D:/AI/models/out_of_set_net_v2/final_model/' + 'out_of_set_net_ae'
out_of_set_net_model_tan = 'D:/AI/models/out_of_set_net_v2/final_model/' + 'out_of_set_net_tan'
out_of_set_net_best = 'D:/AI/models/out_of_set_net_v2/final_model/' + 'out_of_set_net_best'


In [7]:
#Reset the graph 
reset_graph()

#Set constants for Neural Network
dropout_rate = 0.3
learning_rate = 1
n_hidden1 = 2000
n_hidden2 = 1000
n_hidden3 = 500
n_hidden4 = 100
n_final_layer = 2

#Size of final Layer after CNN
cnn_code_layer_size = 4320

#For Decay
likely_batch_size = 30


"""
#Placeholder for input data
X = tf.placeholder(tf.float32, shape=[None, 331, 331, 3], name="input")
y = tf.placeholder(tf.float32, shape=[None,8], name="bounding_box")
"""

filename = tf.placeholder(tf.string, shape=[None], name="tf_records")
batch_size = tf.placeholder(tf.int64, shape=[], name= "Batch_Size")
num_epochs = tf.placeholder(tf.int64, shape=[], name= "Num_epochs")
training = tf.placeholder_with_default(True, shape=(), name = 'training')
handle = tf.placeholder(tf.string, shape=[], name="Dataset")

training_set = build_dataset(True, filename, batch_size, num_epochs, num_parallel_calls=8)
val_set = build_dataset(False, filename, batch_size, num_epochs, num_parallel_calls=8)

train_iterator = training_set.make_initializable_iterator()
val_iterator = val_set.make_initializable_iterator()



iterator = tf.data.Iterator.from_string_handle(
    handle, training_set.output_types, training_set.output_shapes)
next_data = iterator.get_next()
X, y, file = next_data


#Define initalizer and batch normalization layers
bn_batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=0.9)
he_init = tf.contrib.layers.variance_scaling_initializer()


#Import the Nas_Net and build it
with slim.arg_scope(nas.pnasnet_large_arg_scope()):
    net, end_points = nas.build_pnasnet_large(X, num_classes=1001, is_training=False)
    
    #A saver to load the pretrained Data
    saver = tf.train.Saver(name="Original_Saver")
    
    #For getting predictions from Original Network
    pnas_net_predictions = tf.get_default_graph().get_tensor_by_name("final_layer/predictions:0")
    
    #get indicies for doing reduction
    indices = tf.get_default_graph().get_tensor_by_name("final_layer/Mean/reduction_indices:0")
    
    #Load in the noder where we are going to connect our own network
    last_feature_node = tf.get_default_graph().get_tensor_by_name("cell_11/cell_output/concat:0")


with tf.name_scope("Out_Of_Set_Classifier"):
    #Use a stop layer to freeze all the layers beneath in Nas-Net
    
    with tf.name_scope("Isolate_Image_Features"):
        #get the output of the last cell layer

        starting_relu = tf.nn.relu(last_feature_node, name="first_relu")
        mean_pool = tf.reduce_mean(starting_relu, indices, name="condensing_mean")
        cnn_code_layer = tf.stop_gradient(mean_pool)
    
    with tf.name_scope("Autoencoder"):
    
        with tf.name_scope("Hidden_Layer_forward_1"):
            hidden1 = tf.layers.dense(cnn_code_layer, n_hidden1, name="hidden1", kernel_initializer=he_init)
            bn1 = bn_batch_norm_layer(hidden1)
            bn1_act = tf.nn.relu(bn1)

        with tf.name_scope("Hidden_Layer_forward_2"):
            hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2", kernel_initializer=he_init)
            bn2 = bn_batch_norm_layer(hidden2)
            bn2_act = tf.nn.relu(bn2)

        with tf.name_scope("Hidden_Layer_forward_3"):
            hidden3 = tf.layers.dense(bn2_act, n_hidden3, name="hidden3", kernel_initializer=he_init)
            bn3 = bn_batch_norm_layer(hidden3)
            bn3_act = tf.nn.relu(bn3) 
            
        with tf.name_scope("Hidden_Layer_forward_4"):
            hidden4 = tf.layers.dense(bn3_act, n_hidden4, name="hidden4", kernel_initializer=he_init)
            bn4 = bn_batch_norm_layer(hidden4)
            AutoCode = tf.nn.relu(bn4, name="Autoencoder_Value")
            AutoOutput = tf.stop_gradient(AutoCode, name="Autoencoder_Output")
            
            
        with tf.name_scope("Hidden_Layer_Reverse_1"):
            hidden5 = tf.layers.dense(AutoCode, n_hidden3, name="hidden5", kernel_initializer=he_init)
            bn5 = bn_batch_norm_layer(hidden5)
            bn5_act = tf.nn.relu(bn5) 
            
        with tf.name_scope("Hidden_Layer_Reverse_2"):
            hidden6 = tf.layers.dense(bn5_act, n_hidden2, name="hidden6", kernel_initializer=he_init)
            bn6 = bn_batch_norm_layer(hidden6)
            bn6_act = tf.nn.relu(bn6) 
            
        with tf.name_scope("Hidden_Layer_Reverse_3"):
            hidden7 = tf.layers.dense(bn6_act, n_hidden1, name="hidden7", kernel_initializer=he_init)
            bn7 = bn_batch_norm_layer(hidden7)
            bn7_act = tf.nn.relu(bn7)
        
        with tf.name_scope("Reconstruction_Layer_Final"):
            final_reconstruction_layer = tf.layers.dense(bn7_act, cnn_code_layer_size, name="Reconstruction_Layer", 
                                                         kernel_initializer=he_init)

            
        with tf.name_scope("AutoEncoder_Loss"):
            ae_loss = tf.losses.mean_squared_error(cnn_code_layer,final_reconstruction_layer)
            ae_loss_summary = tf.summary.scalar('ae_loss_summary', ae_loss)
            
           
        with tf.name_scope("AutoEncoder_Train"):
            
            ae_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
            ae_extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            
            ae_global_step = tf.Variable(0, trainable=False, name='ae_global_step')
            
            with tf.control_dependencies(ae_extra_update_ops):
                training_op_ae = ae_optimizer.minimize(ae_loss, global_step=ae_global_step)
 
    saver_ae = tf.train.Saver(name="AutoEncoder_Saver")
    
    with tf.name_scope("Classifier"):
        
        with tf.name_scope("Class_Hidden_Layer_1"):
            hidden1_cat = tf.layers.dense(cnn_code_layer, n_hidden1, name="hidden1_cat", kernel_initializer=he_init)
            hidden1_drop = tf.layers.dropout(hidden1_cat, dropout_rate, training=training)
            bn1_cat = bn_batch_norm_layer(hidden1_drop)
            bn1_act_cat = tf.nn.relu(bn1_cat)

        with tf.name_scope("Class_Hidden_Layer_2"):
            hidden2_cat = tf.layers.dense(bn1_act_cat, n_hidden2, name="hidden2_cat", kernel_initializer=he_init)
            hidden2_drop = tf.layers.dropout(hidden2_cat, dropout_rate, training=training)
            bn2_cat = bn_batch_norm_layer(hidden2_drop)
            bn2_act_cat = tf.nn.relu(bn2_cat)

        with tf.name_scope("Class_Hidden_Layer_3"):
            hidden3_cat = tf.layers.dense(bn2_act_cat, n_hidden3, name="hidden3_cat", kernel_initializer=he_init)
            hidden3_drop = tf.layers.dropout(hidden3_cat, dropout_rate, training=training)
            bn3_cat = bn_batch_norm_layer(hidden3_drop)
            bn3_act_cat = tf.nn.relu(bn3_cat) 

        with tf.name_scope("Class_Hidden_Layer_4"):
            hidden4_cat = tf.layers.dense(bn3_act_cat, n_hidden4, name="hidden4_cat", kernel_initializer=he_init)
            hidden4_drop = tf.layers.dropout(hidden4_cat, dropout_rate, training=training)
            bn4_cat = bn_batch_norm_layer(hidden4_drop)
            bn4_act_cat = tf.nn.relu(bn4_cat)     
        
        with tf.name_scope("Final_Layer"): 
            logits_before_bn = tf.layers.dense(bn4_act_cat, n_final_layer, name="outputs")
            logits = bn_batch_norm_layer(logits_before_bn, name="logits")
            softmax = tf.nn.softmax(logits, name="final_soft_max")
            max_val = tf.argmax(softmax,axis=1,name="softmax_Category_int",output_type=tf.int32)
            stop_max = tf.stop_gradient(max_val, name="Stop_Max")


        with tf.name_scope("loss"):
                xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
                loss_cat = tf.reduce_mean(xentropy, name="loss_cat")
                loss_summary_cat = tf.summary.scalar('loss_summary_cat', loss_cat)
            
        with tf.name_scope("train"):
            global_step = tf.Variable(0, trainable=False, name='global_step')
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

            extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

            with tf.control_dependencies(extra_update_ops):
                training_op = optimizer.minimize(loss_cat, global_step=global_step)


        with tf.name_scope("eval"):
            correct = tf.nn.in_top_k(logits, y, 1)
            accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
            accuracy_summary = tf.summary.scalar('accuracy_summary', accuracy)
    
    saver_class = tf.train.Saver(name="Class_Saver")
            
    with tf.name_scope("Tan"):
        
        with tf.name_scope("Sort_to_correct_Tan"):
            y_true = tf.to_float(tf.expand_dims(y,1))
            y_guess = tf.to_float(tf.expand_dims(stop_max,1))
            conditional = tf.cond(training, lambda: y_true, lambda: y_guess)
            
        
        with tf.name_scope("Tan_Network"):
            tan_loss, tan_likelihoods, samp = tan_cond(AutoOutput, conditional)

            
        with tf.name_scope("Tan_Trainer"):
            likelihood_loss_summary = tf.summary.scalar('likelihood_loss_summary_class', tan_loss)
            
            global_step_tan = tf.Variable(0, trainable=False, name="global_step_tan")
            
            learning_rate = tf.train.exponential_decay( 
                learning_rate=0.001, 
                global_step=global_step_tan,
                decay_steps=int( ( 50000 / ( 2 * likely_batch_size ) ) ), 
                decay_rate=0.99, 
                staircase=True
            )
            tan_train_op = tf.train.RMSPropOptimizer( learning_rate ).minimize( tan_loss )
                 
            
#Variables for global initialization
saver_tan = tf.train.Saver(name="Final_Saver")
init = tf.global_variables_initializer()

INFO:tensorflow:A GPU is available on the machine, consider using NCHW data format for increased speed on GPU.
Building tan Graph,
	conditioning Tensor("Out_Of_Set_Classifier/Tan/Sort_to_correct_Tan/cond/Merge:0", shape=(?, 1), dtype=float32)
[<function log_rescale.<locals>.invmap at 0x000001B407C21620>, <function log_rescale.<locals>.invmap at 0x000001B407C21400>, <function log_rescale.<locals>.invmap at 0x000001B407C210D0>, <function log_rescale.<locals>.invmap at 0x000001B406277AE8>, <function log_rescale.<locals>.invmap at 0x000001B406277D90>, <function log_rescale.<locals>.invmap at 0x000001B406277B70>, <function log_rescale.<locals>.invmap at 0x000001B4062778C8>, <function log_rescale.<locals>.invmap at 0x000001B4062776A8>, <function get_LU_map.<locals>.invmap at 0x000001B406277510>]


In [8]:
#Write Graph to log directory along with checkpoints
filewriter = tf.summary.FileWriter(model_path, tf.get_default_graph())

In [9]:
#Initialize all variables and restore the lower layer
with tf.Session() as sess:
    #Initalizer all variables
    init.run()
    
    #Restore the pretrained variables from Nas-Net
    saver.restore(sess, nas_net_model)
    
    
    #Save all of these variables to the new Cell_Net Model
    saver_class.save(sess, out_of_set_net_model_class)
    saver_ae.save(sess, out_of_set_net_model_ae)
    saver_tan.save(sess, out_of_set_net_model_tan)

INFO:tensorflow:Restoring parameters from D:/AI/models/pnas_net/model.ckpt


# Train the Network

Here we are going to train the network. Accuracy/Loss is recorded
Note for this version, a certain amount of the data is seen every training step. 
set train_size for how many images are trained on in each epoch
set batch_size for how many images are trained at once.
num_epochs is how many times the network sees that ammount of training data


# Train the AutoEncoder

We will Begin by training the autoencoder on all the data in our training set.

In [ ]:
#For doing the initial training
#Total number of epochs to train
epochs = 100
steps_between_test_save = 1
batch = 30
train_size = 4444
all_data_steps = np.int(np.floor(train_size/batch))
lowest_loss = 10000


with tf.Session() as sess:
    start_time = time.time()
    #restore saver, build iterator, set the step to the global step
    saver_ae.restore(sess, out_of_set_net_model_ae)
    
    #Set up the global steps
    step = 0
    print("Loaded model. Training network initially. Logs into: " + model_path)
    
    #initialize iterator
    sess.run(train_iterator.initializer, feed_dict={filename: train_list, batch_size: batch, num_epochs:epochs})
    sess.run(val_iterator.initializer, feed_dict={filename: val_list, batch_size: batch, num_epochs:epochs})
    train_handle = sess.run(train_iterator.string_handle())
    val_handle = sess.run(val_iterator.string_handle())
    
    
    #Iterate through training 
    while step < epochs:
        for i in range(all_data_steps):

            #run Training Op
            sess.run([training_op_ae], feed_dict={handle: train_handle})
        
        #see if we are improving on the test data
        #Maybe Test Accuracy
        if ((step % steps_between_test_save) == 0 and step != 0) :
            loss_sum, loss_val = sess.run([ae_loss_summary, ae_loss], feed_dict = {handle: val_handle, training: False})
            filewriter.add_summary(loss_sum, step)
            print("Epoch: " + str(step) + " Loss: " + str(loss_val))
            saver_ae.save(sess, out_of_set_net_model_ae)
        step = step + 1
            
    #Finish the final Model
    saver_ae.save(sess, out_of_set_net_model_ae)
    end_time = time.time()
    total_steps = tf.train.global_step(sess, ae_global_step)
    final_time = end_time - start_time
    print("Did " + str(total_steps) + " of loss minimized training in " + str(final_time) + " seconds.")

INFO:tensorflow:Restoring parameters from D:/AI/models/out_of_set_net_v2/final_model/out_of_set_net_ae
Loaded model. Training network initially. Logs into: D:/AI/models/out_of_set_net_v2/logs/out_of_set_v2_tan-run-20181130174101/
Epoch: 1 Loss: 0.0376788
Epoch: 2 Loss: 0.0304208
Epoch: 3 Loss: 0.0340711
Epoch: 4 Loss: 0.0352295
Epoch: 5 Loss: 0.0350073
Epoch: 6 Loss: 0.0295109
Epoch: 7 Loss: 0.0314069
Epoch: 8 Loss: 0.02947
Epoch: 9 Loss: 0.0347589
Epoch: 10 Loss: 0.0275023
Epoch: 11 Loss: 0.0316035
Epoch: 12 Loss: 0.0379067


# Train the classifier

Here we are going to train it to recognize the two classes of images that we have for that data.

In [ ]:
epochs = 100
steps_between_test_save = 1
batch = 30
train_size = 4444
all_data_steps = np.int(np.floor(train_size/batch))
lowest_loss = 10000


with tf.Session() as sess:
    start_time = time.time()
    #restore saver, build iterator, set the step to the global step
    saver_ae.restore(sess, out_of_set_net_model_ae)
    
    #Set up the global steps
    step = 0
    print("Loaded model. Training network initially. Logs into: " + model_path)
    
    #initialize iterator
    sess.run(train_iterator.initializer, feed_dict={filenames: train_list, batch_size: batch, num_epochs:epochs})
    sess.run(val_iterator.initializer, feed_dict={filenames: val_list, batch_size: batch, num_epochs:epochs})
    train_handle = sess.run(train_iterator.string_handle())
    val_handle = sess.run(val_iterator.string_handle())
    
    #Iterate through training 
    while step < epochs:
        for i in range(all_data_steps):

            #run Training Op
            sess.run([training_op], feed_dict={handle: train_handle})
        
        #see if we are improving on the test data
        #Maybe Test Accuracy
        if ((step % steps_between_test_save) == 0 and step != 0) :
            loss_sum, loss_val, acc_sum = sess.run([loss_summary_cat, loss_cat, accuracy_summary], 
                                                   feed_dict = {handle: val_handle ,training: False})
            filewriter.add_summary(loss_sum, step)
            filewriter.add_summary(acc_sum, step)
            print("Epoch: " + str(step) + " Loss: " + str(loss_val))
            saver_class.save(sess, out_of_set_net_model_class)

        step = step + 1
            
    #Finish the final Model
    saver_class.save(sess, out_of_set_net_model_class)
    end_time = time.time()
    total_steps = tf.train.global_step(sess, global_step)
    final_time = end_time - start_time
    print("Did " + str(total_steps) + " of loss minimized training in " + str(final_time) + " seconds.")


# Train theTAN

Next we train the TAN to get the proper likelihood

In [ ]:
#For doing the initial training
#Total number of epochs to train
epochs = 200
steps_between_test_save = 1
batch = 30
train_size = 4444
all_data_steps = np.int(np.floor(train_size/batch))
lowest_like = 10000


with tf.Session() as sess:
    start_time = time.time()
    #restore saver, build iterator, set the step to the global step
    saver_class.restore(sess, out_of_set_net_model_class)
    
    #Set up the global steps
    step = 0
    print("Loaded model. Training network initially. Logs into: " + model_path)
    
    #initialize iterator
    sess.run(train_iterator.initializer, feed_dict={filenames: train_list, batch_size: batch, num_epochs:epochs})
    sess.run(val_iterator.initializer, feed_dict={filenames: val_list, batch_size: batch, num_epochs:epochs})
    train_handle = sess.run(train_iterator.string_handle())
    val_handle = sess.run(val_iterator.string_handle())
    

    #Iterate through training 
    while step < epochs:
        for i in range(all_data_steps):

            #run Training Op
            sess.run([tan_train_op], feed_dict={handle: train_handle})
        
        #see if we are improving on the test data
        #Maybe Test Accuracy
        if ((step % steps_between_test_save) == 0 and step != 0) :
            like_sum, like_val = sess.run([likelihood_loss_summary, tan_loss], feed_dict = {handle: val_handle, training: False})
            filewriter.add_summary(like_sum, step)
            print("Epoch: " + str(step) + " Loss: " + str(like_val))
            saver_tan.save(sess, out_of_set_net_model_tan)
            if lowest_like > like_val:
                lowest_like = like_val
                saver_tan.save(sess, out_of_set_net_best)
        step = step + 1
            
    #Finish the final Model
    saver_tan.save(sess, out_of_set_net_model_tan)
    end_time = time.time()
    total_steps = tf.train.global_step(sess, global_step_tan)
    final_time = end_time - start_time
    print("Did " + str(total_steps) + " of loss minimized training in " + str(final_time) + " seconds.")

# Do metric testing

Here we will run through all of the training data and relate accuracy with confidence

In [ ]:
out_of_set_net_model = 'D:/AI/models/out_of_set_net_v2/final_model/' + 'out_of_set_net'

batch = 1
epochs = 1

#Set up frame
column_list = ['Item_Number', 'file', 'Correct_Category', 'Correct_Softmax_Value', 'Correct_Softmax_Index','Estimated_Category','Estimated_Category_Index',
                                       'Estimated_Category_Strength','Max_Confidence_Index_Value', 'Calculated_Confidence',
                                       'Correct_Confidence','Confidence_Score']
full_data_frame = pd.DataFrame(columns=column_list)



with tf.Session() as sess:
    
    sess.run(iterator.initializer, feed_dict={filenames: train_list, batch_size: batch, num_epochs:epochs})
    
    #Get Saver Data
    new_saver = tf.train.import_meta_graph(out_of_set_net_model + '.meta')
    new_saver.restore(sess, out_of_set_net_model)
    
    #Set up environment for test
    training = tf.get_default_graph().get_tensor_by_name("training:0")
    soft_max_pna = tf.get_default_graph().get_tensor_by_name("final_layer/predictions:0")
    soft_max_confidence = tf.get_default_graph().get_tensor_by_name("Out_Of_Set_Classifier/Final_Layer/final_soft_max:0")
    
    
    #Loop Through Test Data
    
    for i in range(epochs):
        X_val, y_val, filename, y_category, y_confidence, logs = sess.run([X, y, file, soft_max_pna, soft_max_confidence, logits], 
                                                                   feed_dict={training:False})
        file_string = filename[0].decode("utf-8")
        
        #correct_class = get_ground_truth_string(file_string, validation_array, class_list=old_image_net)
        #correct_imagenet_class_num = find_new_imagenet_ground_truth_int(correct_class, image_net_dict_file)

        correct_class = 'Dog'
        correct_imagenet_class_num = 5

        y_confidence_correct = y_val[[0]]
        
        #Append Data Frame with Requried Information
        cat_max = np.argmax(y_category)
        con_max = np.argmax(y_confidence)

        case_number = i
        item_name = image_net_dict_file.get(cat_max - 1)
        y_category_strength = y_category[[0],[cat_max]][0]
        y_confidence_strength = y_confidence[[0],[con_max]][0]
        y_confidence_correct = y_val[[0]][0]
        confidence_score = y_confidence[[0],[0]][0]
        correct_softmax_value = y_category[[0],correct_imagenet_class_num + 1][0]

        to_add = pd.DataFrame([[case_number,file_string,correct_class,correct_softmax_value,correct_imagenet_class_num,item_name,cat_max - 1,y_category_strength,con_max,y_confidence_strength,
                            y_confidence_correct,confidence_score]], columns = column_list)
        full_data_frame = full_data_frame.append(to_add)

In [ ]:
#Print out the dataframe

full_data_frame.to_csv('Out_Of_Set_Test_Run_1.csv', sep=',')

# Additional Testing

Here are some helpful scripts for doing additional testing

In [ ]:
#Location and models to use for testing

nas_net_model = 'D:/AI/models/pnas_net/model.ckpt'
out_of_set_net_logs = 'D:/AI/models/out_of_set_net_v2/logs'
out_of_set_net_model = 'D:/AI/models/out_of_set_net_v2/final_model/' + 'out_of_set_net'
out_of_set_net_best = 'D:/AI/models/out_of_set_net_v2/final_model/' + 'out_of_set_net_best'

In [ ]:
#Test a single test image
offset_correction = 0

with tf.Session() as sess:
    #Restore File
    saver2.restore(sess, out_of_set_net_model)
    
    #Pull sample from validation set
    sess.run(iterator.initializer, feed_dict={filenames: val_list_all, batch_size: 30, num_epochs:1})
    
    #
    like, y_val = sess.run([tan_likelihood_0, y], feed_dict={training: False})
                    

In [ ]:
like

In [ ]:
X_val = item[0]
Y_1 = item[1]
Y_2 = item[2]
Y_3 = item[3]
Y_4 = item[4]
file = item[5]

In [ ]:
Y_val = np.array([])
Y_val = np.append(Y_val,Y_1)
Y_val = np.append(Y_val,Y_2)
Y_val = np.append(Y_val,Y_3)
Y_val = np.append(Y_val,Y_4)
Y_val

In [ ]:
#display an example and label

x_val = X_val.reshape(331,331,3)
y_val = Y_val
x_array = y_val[0:8:2] 
y_array = y_val[1:8:2]

y_est = Y_estimate[0]
x_guess = y_est[0:8:2] 
y_guess = y_est[1:8:2]
y_est

In [ ]:
#Correct Answer
from matplotlib import pyplot as plt
plt.imshow(x_val, interpolation='nearest')
plt.scatter(x=x_array, y=y_array, c='r', s=40)
plt.axis('on')
plt.show()

In [ ]:
#Guess
from matplotlib import pyplot as plt
plt.imshow(x_val, interpolation='nearest')
plt.scatter(x=x_guess, y=y_guess, c='r', s=40)
plt.axis('on')
plt.show()

In [ ]:
loss_val

In [ ]:
with tf.Session() as sess:
    #restore graph from meta and restore variables
    new_saver = tf.train.import_meta_graph(out_of_set_net_best + '.meta')
    new_saver.restore(sess, out_of_set_net_best)
    soft = tf.get_default_graph().get_tensor_by_name("Out_Of_Set_Classifier/Final_Layer/final_soft_max:0")
    input_tensor = tf.get_default_graph().get_tensor_by_name("input:0")
    val = soft.eval(feed_dict={input_tensor: cat, training: False})

In [ ]:
current_step


In [ ]:
image_net_dict_file

In [ ]:
frame

In [ ]:
list = np.reshape(last_layers, (num_epochs,batch_size,4320))

In [ ]:
list

In [ ]:
with tf.Session() as sess:
    #restore saver, build iterator, set the step to the global step
    saver2.restore(sess, out_of_set_net_model)
    
    #Set up the global steps
    total_steps = tf.train.global_step(sess, global_step)
    
    print("Did " + str(total_steps) + " of loss minimized training in " + str(final_time) + " time.")
    print("Done!")

In [ ]:
final_time